In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

In [ ]:
from IPython.display import display, Javascript

display(Javascript('''
(function() {
    var interval = setInterval(function() {
        if (typeof google !== 'undefined' && google.translate && google.translate.TranslateElement) {
            clearInterval(interval);
            google.translate.TranslateElement = function() {};
            document.getElementById('google_translate_element')?.remove();
        }
    }, 1000);
})();
'''))

## Update repository

In [ ]:
! git pull

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
import multiprocessing
from pathlib import Path
from collections import OrderedDict
import logging

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

## Initialize Path

In [ ]:
PATH = Path('data')
model_dir = PATH / 'models'
model_path = model_dir / 'simple_cnn_mnist_model.ckpt'
config_dir = PATH / 'config'
config_dir.mkdir(exist_ok=True, parents=True)
config_1_layer_path = config_dir / 'neurons_1_layer.json'
config_cnn_layer_1 = config_dir / 'neurons_cnn_1_layer.json'
images_dir = PATH / 'images'
images_dir.mkdir(exist_ok=True, parents=True)
pumpkin_path = PATH / 'Pumpkin_Seeds_Dataset.xlsx'

## Load the model

In [ ]:
model = torch.load(model_path, map_location='cpu')

In [ ]:
def clear_state_dict(state_dict):
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.0.', 'conv1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.3.', 'conv2.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.8.', 'fc1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.11.', 'fc2.')] = state_dict.pop(key)

    return state_dict

In [ ]:
state_dict = clear_state_dict(model['state_dict'])

In [ ]:
net = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)),
    ('act1', nn.ReLU()),
    ('mxp1', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('conv2', nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)),
    ('act2', nn.ReLU()),
    ('mxp2', nn.MaxPool2d(kernel_size=2, stride=2)),
    ('flatten', nn.Flatten()),
    ('fc1', nn.Linear(64 * 7 * 7, 128)),
    ('act3', nn.ReLU()),
    ('fc2', nn.Linear(128, 10)),
]))

In [ ]:
net.load_state_dict(state_dict)

In [ ]:
net = net.eval()

## Initialize MNIST dataset

In [ ]:
transform = transforms.Compose(
            [
                ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
)

In [ ]:
data_train = MNIST(images_dir, train=True, download=True)
data_test = MNIST(images_dir, train=False, download=True)

In [ ]:
next(net.parameters()).device

In [ ]:
device = find_device()
device

In [ ]:
wnet = NetWrapper(net, transform)

In [ ]:
wnet.net

In [ ]:
wnet.net[:6]

In [ ]:
layer_V_n = 3
layer_U_n = 6

In [ ]:
wnet.device

In [ ]:
bs = 8

In [ ]:
V_X_train, X_V_train = layer_V(data_train, wnet, k=layer_V_n, bs=bs)

In [ ]:
V_X_test, X_V_test = layer_V(data_test, wnet, k=layer_V_n, bs=bs)

In [ ]:
U_X_train, X_U_train = layer_V(data_train, wnet, k=layer_U_n, bs=bs)

In [ ]:
U_X_test, X_U_test = layer_V(data_test, wnet, k=layer_U_n, bs=bs)

## Sorting vectors

In [ ]:
V_X_digits, V_X_sorteds = sort_V_X(V_X_train, data_train)

## Alanyze maximum stimulus

In [ ]:
v_Ds = dict()
u_Ds = dict()
G_v_tests = dict()
G_u_tests = dict()
uncn_dict = dict()

In [ ]:
i = 9
ths = [
    568, #0
    330, #1
    672, #2
    580, #3
    470, #4
    590, #5
    484, #6
    544, #7
    640, #8
    584  #9
]
v = np.copy(V_X_sorteds[i][ths[i]])

In [ ]:
for i in range(10):
    layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
    G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
        v, 
        V_X_test, 
        U_X_test, 
        data_test
    )
    v_Ds[i] = v_D
    u_Ds[i] = u_D
    G_v_tests[i] = G_v_test
    G_u_tests[i] = G_u_test
    uncn_dict[i] = uncn_reps

In [ ]:
uncn_dict[i]

In [ ]:
show_grid(G_v_tests[i], data_test, nrow=32)

In [ ]:
show_grid(G_u_tests[i], data_test, nrow=32)

In [ ]:
y_hs = np.argmax(wnet(*[data_test[idx][0] for idx in G_u_test]))

In [ ]:
uncn_hat = layer_fca.count_ys(y_hs)

In [ ]:
show_grid(G_u_tests[i], data_test, nrow=48, h=64, w=64, my=i)

## Experiments with shapes

In [ ]:
# Example usage
dataset = CustomShapeDataset(num_samples=1000)
# dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

shapes_shp = get_digits(dataset)

data_digits = get_digits(data_train)

# Get a batch of images
images, labels = zip(*[(x, y) for idx_l, (x, y) in enumerate(dataset) if idx_l < 32])

asgn_max = True

# Show a batch of images
show_images(images[:16], labels[:16])

In [ ]:
ring = np.copy(shapes_shp['nring'][0][0].numpy())
ring[-13 :, :] = 0
plt.imshow(ring)

In [ ]:
v_X_shapes_c = wnet(ring, k=layer_V_n)
visualize_slices(v_X_shapes_c[0])

In [ ]:
c_idx = 0
max_index, _, v_c = set_v(v_X_shapes_c[c_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_c[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_c, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_X_shapes_v = np.array(wnet(*shapes_shp['vertical_line'], k=layer_V_n))
v_X_shapes_h = np.array(wnet(*shapes_shp['horizontal_line'], k=layer_V_n))
v_X_shapes_e = np.array(wnet(*shapes_shp['sring'], k=layer_V_n))
v_X_shapes_r = np.array(wnet(*shapes_shp['nring'], k=layer_V_n))
v_X_shapes_v.shape, v_X_shapes_h.shape, v_X_shapes_e.shape, v_X_shapes_r.shape

In [ ]:
i = 3

In [ ]:
show_img(shapes_shp['vertical_line'], i)

In [ ]:
visualize_slices(v_X_shapes_h[i])

In [ ]:
diag = torch.zeros((28, 28), dtype=torch.float32)
for i in range(12, 18):
    diag[28 - i, i] = 255
diag /= 255
# diag = diag.t()
plt.imshow(diag)

In [ ]:
v_X_shapes_d = wnet(diag.unsqueeze(0), k=layer_V_n)
visualize_slices(v_X_shapes_d[0])

In [ ]:
d_idx = 0
max_index, _, v_d = set_v(v_X_shapes_d[d_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_d[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_d, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
visualize_slices(v_X_shapes_v[0])

In [ ]:
v_idx = 0
max_index, _, v_v = set_v(v_X_shapes_v[v_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_v[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
h_idx = 0
max_index, _, v_h = set_v(v_X_shapes_h[h_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_h[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_h, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
c_idx = 0
max_index, _, v_c = set_v(v_X_shapes_c[c_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_c[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_c, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
e_idx = 0
max_index, _, v_e = set_v(v_X_shapes_e[e_idx], denm=4.0, val_th=0.2, asgn_max=asgn_max)
show_activation(v_e[max_index[0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_e, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_l = [
    v_v,
    v_d, 
    v_h,
]
v = np.max(np.array(v_l), axis=0)
v.shape

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)

In [ ]:
show(data_digits[0][52:65])

In [ ]:
img_indices = [2, 3, 9, 10, 12, 16, 34, 38, 39, 53, 55, 58, 62]
show([data_digits[0][k] for k in img_indices])

In [ ]:
z_idx = 2
max_index, _, v_z = set_vs(*V_X_digits[0][img_indices], denm=2.0, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
v_z[17, 11, 4] = 1 
show_activation(v_z[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_z, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)

In [ ]:
show(data_digits[1][52:65])

In [ ]:
img_indices = [0, 4, 7, 11, 12, 16, 18, 24, 27, 28, 39, 41, 42, 43, 44, 45, 50] #+ [55, 56, 59] #+ [60, 62, 64]
show([data_digits[1][k] for k in img_indices])

In [ ]:
o_idx = 2
max_index, _, v_o = set_vs(*V_X_digits[1][img_indices], denm=2.0, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_o[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_o, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=64)

In [ ]:
show(data_digits[2][26:39])

In [ ]:
img_indices = [0, 1, 5, 22, 24, 27, 28, 29, 31, 32, 33, 36, 37, 38] #+ [2, 3, 7] + [10, 11, 12]
show([data_digits[2][k] for k in img_indices])

In [ ]:
t_idx = 2
max_index, _, v_t = set_vs(*V_X_digits[2][img_indices], denm=5.0, val_th=0.2, asgn_max=asgn_max, verbose=logging.DEBUG)
show_activation(v_t[max_index[0][0]])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_t, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)

In [ ]:
nz_idx_v = gen_line_idx(hv_shift=8, sid=4, eid=9, hv='v')
neurons_v = np.zeros(v_X_shapes_v[v_idx].shape)
neurons_v[fl_v][nz_idx_v] = 0.1
v_v = np.copy(neurons_v)
show_activation(neurons_v[fl_v])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
nz_idx_h = gen_line_idx(hv_shift=3, sid=4, eid=8, hv='h')
neurons_h = np.zeros(v_X_shapes_h[h_idx].shape)
neurons_h[fl_h][nz_idx_h] = 0.1
v_h = np.copy(neurons_h)
show_activation(neurons_h[fl_h])

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v_h, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
v_l = [v_v, v_h]
v = np.max(np.array(v_l), axis=0)
v.shape

In [ ]:
layer_fca = LayerFCA(V_X_train, U_X_train, data_train)
G_v, v_D, u_D, G_u, G_v_test, G_u_test, uncn_reps = layer_fca.find_G_v_us(
    v, 
    V_X_test, 
    U_X_test, 
    data_test
)
print(uncn_reps)
show_grid(G_v_test, data_test, nrow=32)

In [ ]:
show_grid(G_u_test, data_test, nrow=32)